In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("ProductSalesAnalysis") \
        .getOrCreate()

spark

In [2]:
csv_data = """OrderID,Product,Category,Quantity,UnitPrice,Region
1001,Mobile,Electronics,2,15000,North
1002,Laptop,Electronics,1,55000,South
1003,T-Shirt,Apparel,3,500,East
1004,Jeans,Apparel,2,1200,North
1005,TV,Electronics,1,40000,West
1006,Shoes,Footwear,4,2000,South
1007,Watch,Accessories,2,3000,East
1008,Headphones,Electronics,3,2500,North
"""
with open("sales.csv", "w") as f:
    f.write(csv_data)

df = spark.read.csv("sales.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- OrderID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: integer (nullable = true)
 |-- Region: string (nullable = true)

+-------+-------+-----------+--------+---------+------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|
+-------+-------+-----------+--------+---------+------+
|   1001| Mobile|Electronics|       2|    15000| North|
|   1002| Laptop|Electronics|       1|    55000| South|
|   1003|T-Shirt|    Apparel|       3|      500|  East|
|   1004|  Jeans|    Apparel|       2|     1200| North|
|   1005|     TV|Electronics|       1|    40000|  West|
+-------+-------+-----------+--------+---------+------+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import *

df = df.withColumn("TotalPrice", col('Quantity') * col('UnitPrice'))
df.show()

+-------+----------+-----------+--------+---------+------+----------+
|OrderID|   Product|   Category|Quantity|UnitPrice|Region|TotalPrice|
+-------+----------+-----------+--------+---------+------+----------+
|   1001|    Mobile|Electronics|       2|    15000| North|     30000|
|   1002|    Laptop|Electronics|       1|    55000| South|     55000|
|   1003|   T-Shirt|    Apparel|       3|      500|  East|      1500|
|   1004|     Jeans|    Apparel|       2|     1200| North|      2400|
|   1005|        TV|Electronics|       1|    40000|  West|     40000|
|   1006|     Shoes|   Footwear|       4|     2000| South|      8000|
|   1007|     Watch|Accessories|       2|     3000|  East|      6000|
|   1008|Headphones|Electronics|       3|     2500| North|      7500|
+-------+----------+-----------+--------+---------+------+----------+



In [26]:
df.groupBy("Region").agg(sum("TotalPrice").alias("TotalRevenue")).show()

+------+------------+
|Region|TotalRevenue|
+------+------------+
| South|       63000|
|  East|        7500|
|  West|       40000|
| North|       39900|
+------+------------+



In [27]:
df.groupby("Category").agg(sum("TotalPrice").alias("TotalRevenue")).sort(desc("TotalRevenue")).show()

+-----------+------------+
|   Category|TotalRevenue|
+-----------+------------+
|Electronics|      132500|
|   Footwear|        8000|
|Accessories|        6000|
|    Apparel|        3900|
+-----------+------------+



In [29]:
df.groupBy('Region').agg(count('OrderID').alias('OrderCount')).sort(desc('OrderCount')).show(1)

+------+----------+
|Region|OrderCount|
+------+----------+
| North|         3|
+------+----------+
only showing top 1 row



In [31]:
df.groupBy('Category').agg(avg('UnitPrice').alias('AvgUnitPrice')).sort(desc('AvgUnitPrice')).show()

+-----------+------------+
|   Category|AvgUnitPrice|
+-----------+------------+
|Electronics|     28125.0|
|Accessories|      3000.0|
|   Footwear|      2000.0|
|    Apparel|       850.0|
+-----------+------------+



In [33]:
df.filter(col('TotalPrice') > 30000).show()

+-------+-------+-----------+--------+---------+------+----------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|TotalPrice|
+-------+-------+-----------+--------+---------+------+----------+
|   1002| Laptop|Electronics|       1|    55000| South|     55000|
|   1005|     TV|Electronics|       1|    40000|  West|     40000|
+-------+-------+-----------+--------+---------+------+----------+



In [34]:
df = df.withColumn("HighValueOrder", when (col('TotalPrice') > 20000, "Yes" ).otherwise("No"))
df.show()

+-------+----------+-----------+--------+---------+------+----------+--------------+
|OrderID|   Product|   Category|Quantity|UnitPrice|Region|TotalPrice|HighValueOrder|
+-------+----------+-----------+--------+---------+------+----------+--------------+
|   1001|    Mobile|Electronics|       2|    15000| North|     30000|           Yes|
|   1002|    Laptop|Electronics|       1|    55000| South|     55000|           Yes|
|   1003|   T-Shirt|    Apparel|       3|      500|  East|      1500|            No|
|   1004|     Jeans|    Apparel|       2|     1200| North|      2400|            No|
|   1005|        TV|Electronics|       1|    40000|  West|     40000|           Yes|
|   1006|     Shoes|   Footwear|       4|     2000| South|      8000|            No|
|   1007|     Watch|Accessories|       2|     3000|  East|      6000|            No|
|   1008|Headphones|Electronics|       3|     2500| North|      7500|            No|
+-------+----------+-----------+--------+---------+------+-------

In [36]:
df.filter((col('Region') == "North") & (col('HighValueOrder')=="Yes")).show()

+-------+-------+-----------+--------+---------+------+----------+--------------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|TotalPrice|HighValueOrder|
+-------+-------+-----------+--------+---------+------+----------+--------------+
|   1001| Mobile|Electronics|       2|    15000| North|     30000|           Yes|
+-------+-------+-----------+--------+---------+------+----------+--------------+



In [38]:
df.filter(col('HighValueOrder') == "Yes").groupBy('Region').agg(count('OrderID').alias('OrderCount')).show()

+------+----------+
|Region|OrderCount|
+------+----------+
| South|         1|
|  West|         1|
| North|         1|
+------+----------+



In [40]:
highValueDF = df.filter(col('HighValueOrder')=="Yes")
outpath = "/content/drive/MyDrive/high_value_orders.csv"

In [42]:
highValueDF.coalesce(1).write.csv(outpath, header = True, mode = "overwrite")
print("File Saved Successfully")

File Saved Successfully
